In [1]:
import threading
import os
import time

In [2]:
# concurrent/parallel execution
# concurrent means, dependencies exists
# parallel means no dependencies, we will discuss as we go

In [ ]:
# simple thread and thread id


def worker():
    print("=== Inside worker thread ===")
    print("Thread name :", threading.current_thread().name)
    print("Thread ID   :", threading.get_ident())
    print("Process ID  :", os.getpid())
    time.sleep(10) # 10 seconds sleep, 

print("=== Inside main thread ===")
# every python program runs on thread as well, known as main thread
print("Thread name :", threading.current_thread().name)
print("Thread ID   :", threading.get_ident())
print("Process ID  :", os.getpid())

worker() # calling the worker from main thread
t = threading.Thread(target=worker, name="Worker-1")
t.start() # start new thread
print ("waiting thread to complete")
# t.join() # wait for thread to complete

print (" thread completed")

=== Inside main thread ===
Thread name : MainThread
Thread ID   : 8572103744
Process ID  : 69665
=== Inside worker thread ===
Thread name : MainThread
Thread ID   : 8572103744
Process ID  : 69665
=== Inside worker thread ===waiting thread to complete

Thread name : Worker-1
Thread ID   : 6283194368
Process ID  : 69665
 thread completed


In [11]:
# concurrent access probelm

import threading
import time
import random

# Shared resource
restroom_credits = 1

def use_restroom(person_name):
    global restroom_credits

    print(f"{person_name} is trying to enter the restroom\n")

    # Check credit
    if restroom_credits > 0:
        print(f"{person_name} sees credit available {restroom_credits}")

        # Random delay to make race condition visible
        time.sleep(random.uniform(0.5, 2))

        restroom_credits -= 1
        print(f"After sleep,{person_name} ENTERED restroom. Credits left: {restroom_credits}")

        # Simulate restroom usage
        time.sleep(random.uniform(1, 3))

        restroom_credits += 1
        print(f"{person_name} EXITED restroom. Credits restored: {restroom_credits}")
    else:
        print(f"{person_name} could NOT enter (no credits)")

# Create threads
person1 = threading.Thread(target=use_restroom, args=("Person1",))
person2 = threading.Thread(target=use_restroom, args=("Person2",))

# Start threads
person1.start()
person2.start()

# Wait for both to finish
person1.join()
person2.join()

print("Simulation complete")

Person1 is trying to enter the restroom
Person2 is trying to enter the restroom

Person2 sees credit available 1

Person1 sees credit available 1
After sleep,Person2 ENTERED restroom. Credits left: 0
After sleep,Person1 ENTERED restroom. Credits left: -1
Person1 EXITED restroom. Credits restored: 0
Person2 EXITED restroom. Credits restored: 1
Simulation complete


In [ ]:
# with lock

import threading
import time
import random

restroom_credits = 1
lock = threading.Lock()

def use_restroom(person_name):
    global restroom_credits

    print(f"{person_name} is trying to enter the restroom")

    # Lock make the other thread to wait 
    with lock:
        if restroom_credits > 0:
            print(f"{person_name} got the lock and ENTERED")

            restroom_credits -= 1
        else:
            print(f"{person_name} could NOT enter (no credits)")
            return

    # Restroom usage happens outside lock
    time.sleep(random.uniform(1, 3))

    with lock:
        restroom_credits += 1
        print(f"{person_name} EXITED restroom. Credits restored: {restroom_credits}")

# Threads
person1 = threading.Thread(target=use_restroom, args=("Person1",))
person2 = threading.Thread(target=use_restroom, args=("Person2",))


person1.start()
person2.start()

person1.join()
person2.join()
# to wait for the main function/ thread to wait for the smaller/worker function  

print("Simulation complete")
# person2.join()

Person1 is trying to enter the restroom
Person1 got the lock and ENTERED
Person2 is trying to enter the restroom
Person2 could NOT enter (no credits)
Simulation complete


Person1 EXITED restroom. Credits restored: 1


In [20]:
# Producer Threads  --->  Queue  --->  Consumer Thread
""" payload example
{
  "order_id": 1001,
  "amount": 2499.50,
  "items_count": 3
}
"""

' payload example\n{\n  "order_id": 1001,\n  "amount": 2499.50,\n  "items_count": 3\n}\n'

In [21]:
import threading
import queue
import time
import random
import json

# Shared queue (thread-safe)
order_queue = queue.Queue()

# Sentinel to stop consumer
STOP_SIGNAL = object()

# -------------------------
# Producer (Order Creator)
# -------------------------
def producer(name, order_count):
    for _ in range(order_count):
        order = {
            "order_id": random.randint(1000, 9999),
            "amount": round(random.uniform(100, 5000), 2),
            "items_count": random.randint(1, 10)
        }

        print(f"[PRODUCER-{name}] Created order: {json.dumps(order)}")

        order_queue.put(order)

        # Random delay
        time.sleep(random.uniform(0.5, 2))

    print(f"[PRODUCER-{name}] Finished producing orders")


# -------------------------
# Consumer (Order Processor)
# -------------------------
def consumer():
    while True:
        order = order_queue.get()

        if order is STOP_SIGNAL:
            print("[CONSUMER] Stop signal received. Shutting down.")
            break

        print(f"[CONSUMER] Processing order {order['order_id']} "
              f"(Amount: {order['amount']}, Items: {order['items_count']})")

        # Simulate processing time
        time.sleep(random.uniform(1, 3))

        print(f"[CONSUMER] Completed order {order['order_id']}")

        order_queue.task_done()


# -------------------------
# Thread Setup
# -------------------------
producer1 = threading.Thread(target=producer, args=("A", 3))
producer2 = threading.Thread(target=producer, args=("B", 3))
consumer_thread = threading.Thread(target=consumer)

# Start threads
consumer_thread.start()
producer1.start()
producer2.start()

# Wait for producers to finish
producer1.join()
producer2.join()

# Wait until all orders are processed
order_queue.join()

# Stop consumer
order_queue.put(STOP_SIGNAL)
consumer_thread.join()
 

print("Order processing system shut down cleanly.")

[PRODUCER-A] Created order: {"order_id": 1528, "amount": 1469.27, "items_count": 7}
[CONSUMER] Processing order 1528 (Amount: 1469.27, Items: 7)
[PRODUCER-B] Created order: {"order_id": 7413, "amount": 4220.28, "items_count": 5}
[CONSUMER] Completed order 1528
[CONSUMER] Processing order 7413 (Amount: 4220.28, Items: 5)
[PRODUCER-A] Created order: {"order_id": 2041, "amount": 1688.66, "items_count": 4}
[PRODUCER-B] Created order: {"order_id": 4042, "amount": 4899.21, "items_count": 3}
[PRODUCER-A] Created order: {"order_id": 1640, "amount": 2999.23, "items_count": 7}
[PRODUCER-B] Created order: {"order_id": 8730, "amount": 3065.34, "items_count": 1}
[CONSUMER] Completed order 7413
[CONSUMER] Processing order 2041 (Amount: 1688.66, Items: 4)
[PRODUCER-A] Finished producing orders
[PRODUCER-B] Finished producing orders
[CONSUMER] Completed order 2041
[CONSUMER] Processing order 4042 (Amount: 4899.21, Items: 3)
[CONSUMER] Completed order 4042
[CONSUMER] Processing order 1640 (Amount: 2999